## Configurações Iniciais

In [1]:
#Bibliotecas Utilizadas
import IPython as IP
import CoolProp
import CoolProp.CoolProp as CP #Propriedades termodinâmicas
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import psychrolib as psy # Check docmentation at: https://psychrometrics.github.io/psychrolib/api_docs.html
from itertools import product
from scipy.interpolate import interp1d # Biblio para Interpolação

# Configuração Inicial da biblio de psicometria
psy.SetUnitSystem(psy.SI)

## Moiller Diagram for R410A

![](r410a-moiller.png)

## Dados do Compressor

In [2]:
compressor_data = pd.read_csv('fugitsu_compressor_data.csv')

In [3]:
# Colunas da tabela
compressor_data.columns

Index(['Rot', 'Condicao', 'T_evap', 'T_cond', 'T_sup', 'T_sub', 'CAP', 'W',
       'V_cil', 'P_evap', 'P_cond', 'RP', 'h_1', 'h_3', 'm', 'v_1', 'eta_v',
       's_1', 'h_2s', 'W_s', 'eta_g', 'Sigma'],
      dtype='object')

In [4]:
# Amostra dos dados do compressor
compressor_data.head()

,Rot,Condicao,T_evap,T_cond,T_sup,T_sub,CAP,W,V_cil,P_evap,...,h_1,h_3,m,v_1,eta_v,s_1,h_2s,W_s,eta_g,Sigma
0,4800,C,15.0,45.0,20.0,35.0,9572.17,1691.91,0.000015,1.254290e+06,...,431305.922776,256662.500668,0.054810,0.021332,0.974356,1803.36268,452557.513419,1164.795307,0.688450,1.000000
1,3000,C,15.0,45.0,20.0,35.0,5705.38,944.62,0.000015,1.254290e+06,...,431305.922776,256662.500668,0.032669,0.021332,0.929205,1803.36268,452557.513419,694.262622,0.734965,0.625000
2,4000,C,15.0,45.0,20.0,35.0,7876.02,1326.42,0.000015,1.254290e+06,...,431305.922776,256662.500668,0.045098,0.021332,0.962044,1803.36268,452557.513419,958.398266,0.722545,0.833333
3,4800,I,15.0,50.0,20.0,35.0,9565.05,1902.83,0.000015,1.254290e+06,...,431305.922776,256386.384387,0.054683,0.021332,0.972094,1803.36268,455810.126532,1339.952851,0.704189,1.000000
4,4000,I,15.0,50.0,20.0,35.0,7861.63,1507.11,0.000015,1.254290e+06,...,431305.922776,256386.384387,0.044944,0.021332,0.958771,1803.36268,455810.126532,1101.323415,0.730752,0.833333


In [5]:
# Range de RP para cada uma das rotações
print('Range RP para Rotação de 3000. Mínimo: ', 
      compressor_data[compressor_data[ 'Rot'] == 3000]['RP'].min(), 
      ' Máximo: ', compressor_data[compressor_data[ 'Rot'] == 3000]['RP'].max())

print('Range RP para Rotação de 4000. Mínimo: ', 
      compressor_data[compressor_data[ 'Rot'] == 4000]['RP'].min(), 
      ' Máximo: ', compressor_data[compressor_data[ 'Rot'] == 4000]['RP'].max())

print('Range RP para Rotação de 4800. Mínimo: ', 
      compressor_data[compressor_data[ 'Rot'] == 4800]['RP'].min(), 
      ' Máximo: ', compressor_data[compressor_data[ 'Rot'] == 4800]['RP'].max())

Range RP para Rotação de 3000. Mínimo:  2.173446345670816  Máximo:  4.1094359555634234
Range RP para Rotação de 4000. Mínimo:  2.173446345670816  Máximo:  4.1094359555634234
Range RP para Rotação de 4800. Mínimo:  2.173446345670816  Máximo:  4.1094359555634234


In [6]:
# Interpolações para calcular eta_v e eta_g a partir de RP
def eta_from_rp_calc(RP,rot,kind): 
    if kind == 'v' and rot in [3000/60,4000/60,4800/60]:
        try:
            interp_cubica = interp1d(compressor_data[compressor_data['Rot']==rot*60]['RP'], compressor_data[compressor_data['Rot']==rot*60]['eta_v'], kind='cubic')
        except:
            print('Use um valor de RP dentro do intervalo válido!')
    elif kind == 'g' and rot in [3000/60,4000/60,4800/60]:
        try:
            interp_cubica = interp1d(compressor_data[compressor_data['Rot']==rot*60]['RP'], compressor_data[compressor_data['Rot']==rot*60]['eta_g'], kind='cubic')
        except:
            print('Use um valor de RP dentro do intervalo válido!')
    else:
        print("""Insira um valor válido de Rotação do compressor [ 3000,4000,4800] e escolha 
        'v' ou 'g' para escolher qual eficiência calcular""")
        print('RP: ',RP,' rot: ',rot,' kind: ',kind)
        return
    
    return float(interp_cubica(RP))

In [7]:
# Testando a função
eta_from_rp_calc(2.3,3000/60,'v')

0.9217048435199436

## Etapa de Calibração - Encontrar UAs Iniciais a partir dos dados da Whirlpool

### Variáveis Globais - Dados AC Split Inveter da Fujitsu

In [8]:
# Constantes de Mudança de Unidade
CONST_FROM_BTU_TO_SI = 0.293
CONST_FROM_PSI_TO_SI = 6894.76
ATM_PRESSURE = 101325 # SI (N/mˆ2)
CONST_FROM_CELSIUS_TO_K = 273.15

# Líquido Refrigerante utilizando
coolant = 'R410A'

# Parâmetros de fabricação do AC
ROT = [3000/60,4000/60,4800/60] # Vetor com as possíveis rotações do compressor testadas em Hz
VOL_cil = 15e-6 # Volume do cilindo do compressor em mˆ3 
delta_sub = 4.29 # Delta de sub-resfriamento
delta_sup = 3.02 # Delta de super-aquecimento
T_amb = 34.96 + CONST_FROM_CELSIUS_TO_K # Temperatura do ambiente externo (K)
T_int_tbs = 26.7 + CONST_FROM_CELSIUS_TO_K # Temperatura de Bulbo seco do ambiente interno (K)
T_int_tbu = 19.4 + CONST_FROM_CELSIUS_TO_K # Temperatura de Bulbo úmido do ambiente interno (K)

### Função para calibração

In [9]:
# Função para cálculo de UAs partindo das variáveis: T_2, T_cond, T_evap e Q_evap_lat
def CalibrateUAs (T_2,T_cond,T_evap,Q_dot_evap_lat, rot_comp=ROT[2]):
    # Pressões no Evaporador e no Condensador
    P_evap = CP.PropsSI('P','T',T_evap,'Q',1,coolant) # (Pa)
    P_cond = CP.PropsSI('P','T',T_cond,'Q',0,coolant) # (Pa)
    
    # Encontrando as eficiências volumétricas e globais, a partir dos dados do compressor
    PR = P_cond/P_evap # (-)
    
    # Verificando se PR está em um intervalo viável
    if PR >= 2.1734 and PR <= 4.1094:
        pass
    else:
        print('Pressões de Condensão e Evaporação fora do intervalo dos dados disponíveis para o compressor')
        print('Pressão no Condensador: ', P_cond)
        print('Pressão no Evaporador: ', P_evap)
        return 
    
    # Encontrando os valores de Eficiência Gloval e Volumétrica a partir dos dados 
    eta_v = eta_from_rp_calc(PR,rot_comp,'v') # (-)
    eta_g = eta_from_rp_calc(PR,rot_comp,'g') # (-)
    
    # Ponto 1 (Entrada do Compressor)
    T_1 = T_evap + delta_sup # Temperatura no ponto 1 (K)
    v_1 = 1/CP.PropsSI('D','T',T_1,'P',P_evap,coolant) # Densidade Volumétrica do ponto 1 (mˆ3/Kg)
    h_1 = CP.PropsSI('H','T',T_1,'P',P_evap,coolant) # Entalpia real no ponto 1 (J/kg)
    s_1 = CP.PropsSI('S','T',T_1,'P',P_evap,coolant) # Entropia do ponto 1 (J/kg)
    
    # Fluxo do refrigerante
    m_dot = eta_v*rot_comp*VOL_cil/v_1 # Vazão mássica (Kg/s)
    
    # Ponto 2 (Saída do Compressor - Entrada no condensador)
    h_2s = CP.PropsSI('H','P',P_cond,'S',s_1,coolant) # Entalpia Isentrópica do ponto 2 (J/kg)
    Pot = m_dot*(h_2s-h_1)/eta_g # Potência consumida pelo compressor (W)
    h_2a = h_1 + Pot/m_dot # Entalpia adiabática do ponto 2 (J/kg)
    T_2a = CP.PropsSI('T','P',P_cond,'H',h_2a,coolant) # Temperatura no ponto 2a (K)
    
    h_2 = CP.PropsSI('H','T',T_2,'P',P_cond,coolant) # Entalpia real do ponto 2 (J/kg)
    Q_dot_comp = abs(Pot - m_dot*(h_2 - h_1)) # Calor perdido para o ambiente no compressor - BE no refrigerante
    UA_comp = Q_dot_comp/(T_2a-T_amb) # Coef. global de Transferencia de calor X Área do Compressor - BE no exterior
    
    # Ponto 3 (Saída do Condensador)
    T_3 = T_cond - delta_sub # Temperatura no ponto 3
    h_3 = CP.PropsSI('H','T',T_3,'P',P_cond,coolant) # Entalpia real no ponto 3
    Q_dot_cond = m_dot*(h_2-h_3) # Calor perdido para o ambiente no condensador - BE no refrigerante
    UA_cond = Q_dot_cond/(T_cond-T_amb) # Coef. global de Transferencia de calor X Área do Condensador - BE no exterior
    
    # No evaporador
    Q_dot_evap_sens = m_dot*(h_1-h_3) - Q_dot_evap_lat
    UA_evap_sens = Q_dot_evap_sens/(T_int_tbs - T_evap)
    
    
    # Cálculo Psicométricos do Ar no ambiente interior (onde fica instalado o evaporador)
    psy.SetUnitSystem(psy.SI)
    W_interior = psy.GetHumRatioFromTWetBulb(T_int_tbs - CONST_FROM_CELSIUS_TO_K,T_int_tbu - CONST_FROM_CELSIUS_TO_K,ATM_PRESSURE)
    W_evap = psy.GetHumRatioFromRelHum(T_evap - CONST_FROM_CELSIUS_TO_K,1,P_evap)
    
    # Entalpia de Vaporização (Diferença entre a entalpia do valor e o líquido saturado a T_evap)
    hlv = CP.PropsSI('H','P',P_evap,'Q',1,'Water') - CP.PropsSI('H','P',P_evap,'Q',0,'Water')  # (J/kg)
    
    # Por fim, no balanço de energia latente no evaporador
    UA_evap_lat = Q_dot_evap_lat/((W_interior - W_evap)*hlv)
    
    # Armazenando as variáveis calibradas em um dicionário
    variaveis_calibradas = {}
    variaveis_calibradas['eta_v'] = eta_v
    variaveis_calibradas['eta_g'] = eta_g
    variaveis_calibradas['eta_v'] = eta_v
    variaveis_calibradas['eta_g'] = eta_g
    variaveis_calibradas['T_1'] = T_1
    variaveis_calibradas['v_1'] = v_1
    variaveis_calibradas['h_1'] = h_1
    variaveis_calibradas['s_1'] = s_1
    variaveis_calibradas['m_dot'] = m_dot
    variaveis_calibradas['h_2s'] = h_2s
    variaveis_calibradas['Pot'] = Pot
    variaveis_calibradas['h_2a'] = h_2a
    variaveis_calibradas['T_2a'] = T_2a    
    variaveis_calibradas['h_2'] = h_2
    variaveis_calibradas['Q_dot_comp'] = Q_dot_comp
    variaveis_calibradas['UA_comp'] = UA_comp    
    variaveis_calibradas['T_3'] = T_3
    variaveis_calibradas['h_3'] = h_3
    variaveis_calibradas['Q_dot_cond'] = Q_dot_cond
    variaveis_calibradas['UA_cond'] = UA_cond   
    variaveis_calibradas['Q_dot_evap_sens'] = Q_dot_evap_sens
    variaveis_calibradas['UA_evap_sens'] = UA_evap_sens  
    variaveis_calibradas['W_interior'] = W_interior
    variaveis_calibradas['W_evap'] = W_evap
    variaveis_calibradas['hlv'] = hlv
    variaveis_calibradas['UA_evap_lat'] = UA_evap_lat
    
    return variaveis_calibradas

# Calibragem

In [10]:
# Parâmetros auxiliares para calibragem
T_2 = 77.76 + CONST_FROM_CELSIUS_TO_K # Temperatura de descarga do compressor em K
T_evap = 10.86 + CONST_FROM_CELSIUS_TO_K # Temperatura de saída no evaporador em K
T_cond = 45.38 + CONST_FROM_CELSIUS_TO_K # Temperatura no meio do condensador em K 
Q_dot_evap_lat = 6494.32*CONST_FROM_BTU_TO_SI # Calor latente no evaporador em W

In [11]:
variaveis_calibradas = CalibrateUAs(T_2,T_cond,T_evap,Q_dot_evap_lat)

In [14]:
print('Número de variáveis: ',len(variaveis_calibradas))
print('')
variaveis_calibradas

Número de variáveis:  24



{'eta_v': 0.9688481516931082,
 'eta_g': 0.703186354317819,
 'T_1': 287.03,
 'v_1': 0.023801893122229297,
 'h_1': 427913.598729894,
 's_1': 1802.6639715891552,
 'm_dot': 0.04884560131672579,
 'h_2s': 452650.0002766828,
 'Pot': 1718.270555942538,
 'h_2a': 463091.1891295062,
 'T_2a': 343.7721444704031,
 'h_2': 472228.11645543977,
 'Q_dot_comp': 446.29870942245,
 'UA_comp': 12.51463466513754,
 'T_3': 314.23999999999995,
 'h_3': 267989.1488007014,
 'Q_dot_cond': 9976.175187403003,
 'UA_cond': 957.4064479273501,
 'Q_dot_evap_sens': 5908.770162038015,
 'UA_evap_sens': 373.0284193205823,
 'W_interior': 0.011093835138280729,
 'W_evap': 0.0007278330577198207,
 'hlv': 1997782.0091961604,
 'UA_evap_lat': 0.09188442777380339}

## Etapa de Otimização - Resolução do Ciclo pelo Método de Newthon Raphson

In [21]:
def COPCalculator_GivenTemperatures(T_evap,T_cond,UA_comp=UA_comp_calib,UA_cond=UA_cond_calib,
                                     UA_evap_sens=UA_evap_sens_calib,UA_evap_lat=UA_evap_lat_calib,
                                     rot_comp=ROT[0]):
    
    print('')
    print('Valor teste de T_evap: ',T_evap)
    print('Valor teste de T_cond: ',T_cond)
    
    # Pressões de condensação e evaporação
    P_evap = CP.PropsSI('P','T',T_evap,'Q',1,coolant) # (Pa)
    P_cond = CP.PropsSI('P','T',T_cond,'Q',0,coolant) # (Pa)
    
    # Encontrando as eficiências volumétricas e globais, a partir dos dados do compressor
    PR = P_cond/P_evap # (-)
    
    print('')
    print('PR encontrado: ', PR)
    # Verificando se PR está em um intervalo viável
    if PR >= 2.1734 and PR <= 4.1094:
        pass
    else:
        print('Pressões de Condensão e Evaporação fora do intervalo dos dados disponíveis para o compressor')
        print('Pressão no Condensador: ', P_cond)
        print('Pressão no Evaporador: ', P_evap)
        return 
    
    # Encontrando os valores de Eficiência Gloval e Volumétrica a partir dos dados 
    eta_v = eta_from_rp_calc(PR,rot_comp,'v') # (-)
    eta_g = eta_from_rp_calc(PR,rot_comp,'g') # (-)
    
    # Ponto 1 (Entrada do Compressor)
    T_1 = T_evap + delta_sup # Temperatura no ponto 1 (K)
    v_1 = 1/CP.PropsSI('D','T',T_1,'P',P_evap,coolant) # Densidade Volumétrica do ponto 1 (mˆ3/Kg)
    h_1 = CP.PropsSI('H','T',T_1,'P',P_evap,coolant) # Entalpia real no ponto 1 (J/kg)
    s_1 = CP.PropsSI('S','T',T_1,'P',P_evap,coolant) # Entropia do ponto 1 (J/kg)
    
    # Fluxo do refrigerante
    m_dot = eta_v*rot_comp*VOL_cil/v_1 # Vazão mássica (Kg/s)
    
    # Ponto 2 (Saída do Compressor - Entrada no condensador)
    h_2s = CP.PropsSI('H','P',P_cond,'S',s_1,coolant) # Entalpia Isentrópica do ponto 2 (J/kg)
    Pot = m_dot*(h_2s-h_1)/eta_g # Potência consumida pelo compressor (W)
    h_2a = h_1 + Pot/m_dot # Entalpia adiabática do ponto 2 (J/kg)
    T_2a = CP.PropsSI('T','P',P_cond,'H',h_2a,coolant) # Temperatura no ponto 2a (K) 
    Q_dot_comp = UA_comp*(T_2a-T_amb) # Calor perdido para o ambiente no compressor
    
    h_2 = h_1 + ((abs(Pot-Q_dot_comp))/m_dot) # Entalpia real do ponto 2 (J/kg)
    T_2 = CP.PropsSI('T','P',P_cond,'H',h_2,coolant) # Temperatura real do ponto 2
    
    # Ponto 3 (Saída do Condensador)
    T_3 = T_cond - delta_sub # Temperatura no ponto 3
    h_3 = CP.PropsSI('H','T',T_3,'P',P_cond,coolant) # Entalpia real no ponto 3
    Q_dot_cond = m_dot*(h_2-h_3) # Calor perdido para o ambiente no condensador - BE no refrigerante
    T_cond_new = T_amb + Q_dot_cond/UA_cond
    
    return abs(T_cond_new-T_cond)

In [24]:
def COPCalculator(UA_comp=UA_comp_calib,UA_cond=UA_cond_calib,
                  UA_evap_sens=UA_evap_sens_calib,UA_evap_lat=UA_evap_lat_calib,
                  rot_comp=ROT[0]):
    
    # Valores iniciais para T_evap e T_cond
    T_evap_tv = 285
    T_cond_tv = 320
    error = 1000
    n_ite = 0
    
    # Newton Raphson
    print('Progress:',end='')
    while (n_ite < 100):
        try: 
            error = COPCalculator_GivenTemperatures(T_evap=T_evap_tv,T_cond=T_cond_tv,
                          UA_comp=UA_comp_calib,UA_cond=UA_cond_calib,
                          UA_evap_sens=UA_evap_sens_calib,
                          UA_evap_lat=UA_evap_lat_calib,rot_comp=ROT[0])
        except:
            print(error)
            print('Problema no Loop!')
            return
        
        n_ite += 1
        print('.',end='')
        print('')
        print('Erro da Iteração: ', error)
        
        if error < 0.1:
            T_evap = T_evap_tv
            T_cond = T_cond_tv
            break
        else: 
            # Calculando as derivadas parciais em relação as variáveis
            df_dtevap = (COPCalculator_GivenTemperatures(T_evap=T_evap_tv+0.1,T_cond=T_cond_tv,
                              UA_comp=UA_comp_calib,UA_cond=UA_cond_calib,
                              UA_evap_sens=UA_evap_sens_calib,
                              UA_evap_lat=UA_evap_lat_calib,rot_comp=ROT[0]) - error)/(0.1) 
            df_dtcond = (COPCalculator_GivenTemperatures(T_evap=T_evap_tv,T_cond=T_cond_tv+0.01,
                              UA_comp=UA_comp_calib,UA_cond=UA_cond_calib,
                              UA_evap_sens=UA_evap_sens_calib,
                              UA_evap_lat=UA_evap_lat_calib,rot_comp=ROT[0]) - error)/(0.1)

            print('Primeira derivada em relação a T_evap: ', df_dtevap)
            print('Primeira derivada em relação a T_cond: ', df_dtcond)
            # Atualizando os valores das variáveis chutes segundo algoritmo de Newton Raphson
            T_evap_tv = T_evap_tv - error/df_dtevap
            T_cond_tv = T_cond_tv - error/df_dtcond
    
    print('Número de Iterações feitas: ',n_ite)
    print('Erro final: ', error)
        
    return 


#     # Pegando os melhores chutes para as temperaturas T_evap e T_cond
#     T_evap = T_evap_tv
#     T_cond = T_cond_tv
    
#     # Pressões de condensação e evaporação
#     P_evap = CP.PropsSI('P','T',T_evap,'Q',1,coolant) # (Pa)
#     P_cond = CP.PropsSI('P','T',T_cond,'Q',0,coolant) # (Pa)
    
#     # Encontrando as eficiências volumétricas e globais, a partir dos dados do compressor
#     PR = P_cond/P_evap # (-)
    
#     # Verificando se PR está em um intervalo viável
#     if PR >= 2.1734 and PR <= 4.1094:
#         pass
#     else:
#         print('Pressões de Condensão e Evaporação fora do intervalo dos dados disponíveis para o compressor')
#         print('Pressão no Condensador: ', P_cond)
#         print('Pressão no Evaporador: ', P_evap)
#         return 
    
#     # Encontrando os valores de Eficiência Gloval e Volumétrica a partir dos dados 
#     eta_v = eta_from_rp_calc(PR,rot_comp,'v') # (-)
#     eta_g = eta_from_rp_calc(PR,rot_comp,'g') # (-)
    
#     # Ponto 1 (Entrada do Compressor)
#     T_1 = T_evap + delta_sup # Temperatura no ponto 1 (K)
#     v_1 = 1/CP.PropsSI('D','T',T_1,'P',P_evap,coolant) # Densidade Volumétrica do ponto 1 (mˆ3/Kg)
#     h_1 = CP.PropsSI('H','T',T_1,'P',P_evap,coolant) # Entalpia real no ponto 1 (J/kg)
#     s_1 = CP.PropsSI('S','T',T_1,'P',P_evap,coolant) # Entropia do ponto 1 (J/kg)
    
#     # Fluxo do refrigerante
#     m_dot = eta_v*rot_comp*VOL_cil/v_1 # Vazão mássica (Kg/s)
    
#     # Ponto 2 (Saída do Compressor - Entrada no condensador)
#     h_2s = CP.PropsSI('H','P',P_cond,'S',s_1,coolant) # Entalpia Isentrópica do ponto 2 (J/kg)
#     Pot = m_dot*(h_2s-h_1)/eta_g # Potência consumida pelo compressor (W)
#     h_2a = h_1 + Pot/m_dot # Entalpia adiabática do ponto 2 (J/kg)
#     T_2a = CP.PropsSI('T','P',P_cond,'H',h_2a,coolant) # Temperatura no ponto 2a (K) 
#     Q_dot_comp = UA_comp*(T_2a-T_amb) # Calor perdido para o ambiente no compressor
    
#     h_2 = h_1 + ((abs(Pot-Q_dot_comp))/m_dot) # Entalpia real do ponto 2 (J/kg)
#     T_2 = CP.PropsSI('T','P',P_cond,'H',h_2,coolant) # Temperatura real do ponto 2
    
#     # Ponto 3 (Saída do Condensador)
#     T_3 = T_cond - delta_sub # Temperatura no ponto 3
#     h_3 = CP.PropsSI('H','T',T_3,'P',P_cond,coolant) # Entalpia real no ponto 3
#     Q_dot_cond = m_dot*(h_2-h_3) # Calor perdido para o ambiente no condensador - BE no refrigerante
#     T_cond_new = T_amb + Q_dot_cond/UA_cond
    
#     # Psicometria
#     psy.SetUnitSystem(psy.SI)
#     W_interior = psy.GetHumRatioFromTWetBulb(T_int_tbs - CONST_FROM_CELSIUS_TO_K,T_int_tbu - CONST_FROM_CELSIUS_TO_K,ATM_PRESSURE)
#     W_evap = psy.GetHumRatioFromRelHum(T_evap - CONST_FROM_CELSIUS_TO_K,1,P_evap)
#     hlv = CP.PropsSI('H','P',P_evap,'Q',1,'Water') - CP.PropsSI('H','P',P_evap,'Q',0,'Water')  # (J/kg)
 
#     # No evaporador
#     Q_dot_evap_sens = UA_evap_sens*(T_int_tbs - T_evap)
#     Q_dot_evap_lat = UA_evap_lat*(W_interior - W_evap)*hlv
    
#     print('Convergiu!!')
#     print('Valor encontrado para T_evap: ', T_evap)
#     print('Valor encontrado para T_cond: ', T_cond)
    
#     return (Q_dot_evap_sens + Q_dot_evap_lat)/Pot

In [25]:
COPCalculator(UA_cond=500)

Progress:
Valor teste de T_evap:  285
Valor teste de T_cond:  320

PR encontrado:  2.4926744759940203
.
Erro da Iteração:  6.408166086197468

Valor teste de T_evap:  285.1
Valor teste de T_cond:  320

PR encontrado:  2.485417832125673

Valor teste de T_evap:  285
Valor teste de T_cond:  320.01

PR encontrado:  2.4932565305376757
Primeira derivada em relação a T_evap:  -0.17447765518966207
Primeira derivada em relação a T_cond:  0.10623514755252472

Valor teste de T_evap:  321.7277178228445
Valor teste de T_cond:  259.6794165223977

PR encontrado:  0.17165177424105438
Pressões de Condensão e Evaporação fora do intervalo dos dados disponíveis para o compressor
Pressão no Condensador:  508784.68331308046
Pressão no Evaporador:  2964051.4090963188
.
Erro da Iteração:  None


TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Implementação alternativa do método de Newton Raphson

In [26]:
def Newton_system(F, J, x, eps):
    """
    Solve nonlinear system F=0 by Newton's method.
    J is the Jacobian of F. Both F and J must be functions of x.
    At input, x holds the start value. The iteration continues
    until ||F|| < eps.
    """
    F_value = F(x)
    F_norm = np.linalg.norm(F_value, ord=2)  # l2 norm of vector
    iteration_counter = 0
    while abs(F_norm) > eps and iteration_counter < 100:
        delta = np.linalg.solve(J(x), -F_value)
        x = x + delta
        F_value = F(x)
        F_norm = np.linalg.norm(F_value, ord=2)
        iteration_counter += 1

    # Here, either a solution is found, or too many iterations
    if abs(F_norm) > eps:
        iteration_counter = -1
    return x, iteration_counter

In [ ]:
def Main_COPCalculator(UA_comp=UA_comp_calib,UA_cond=UA_cond_calib,
                  UA_evap_sens=UA_evap_sens_calib,UA_evap_lat=UA_evap_lat_calib,
                  rot_comp=ROT[0]):
    
    """ Função para resolver um ciclo termodinâmico representativo de um sistema de refrigeração com 
    N equações e N incógnitas acopladas. Algoritmo baseado no método de Newton Raphson com 
    multi-variáveis. Após a resolucão do problema, é calculado o COP do ciclo para as variáveis de 
    entrada escolhidas.
    
    Variáveis Globais utilizadas:
        T_amb (Temperatura Ambiente em kelvin)
        T_int_tbs (Temperatura de Bulbo Seco para o ambiente Interior em kelvin)
        T_int_tbu (Temperatura de Bulbo Úmido para o ambiente Interior em kelvin)
        coolant (Líquido Refrigerante utilizado)
        VOL_cil (Volume do cilindro do compressor em metros cúbicos)
        delta_sup (Delta de super-aquecimento)
        CONST_FROM_K_TO_C (Constante responsável por mudar a temperatura de Celcius para Kelvin)
        
    Agrs: 
        UA_comp (float): Resistência térmica do Compressor. Valor calibrado como default
        UA_cond (float): Resistência térmica do Condensador. Valor calibrado como default
        UA_evap_sens (float): Resistênca térmica sensível do Evaporador. Valor calibrado como default
        UA_evap_lat (float): Resistência térmica latente do Evaporador. Valor calibrado como default
        
    Returns:
        float: Valor do COP do ciclo.
    
    """
    
    # Variáveis e seus valores inicias
    P_evap = 
    
    # Declaração do vetor com as variáveis
    X = [P_evap, # Pressão do Evaporador (Pa)
         P_cond, # Pressão do Condensador (Pa)
         T_evap, # Temperatura no Evaporador - Saturação (K)
         T_cond, # Temperatura no Condensador - Saturação (K)
         eta_v, # Eficiência Volumétrica do Compressor
         eta_g, # Eficiência Global do Compressor
         T_1, # Temperatura no ponto de sucção (K)
         v_1, # Densidade Volumétrica do ponto de sucção (mˆ3/Kg)
         h_1, # Entalpia real no ponto de sucção (J/kg)
         s_1, # Entropia do ponto de succão (J/kg)
         m_dot, # Vazão mássica (kg/s)
         h_2s, # Entalpia Isentrópica do ponto de descarga (J/kg)
         Pot, # Potência elétrica consumida pelo compressor
         h_2a, # Entalpia Adiabática do ponto de descarga (J/kg)
         T_2a, # Temperatura Adiabática do ponto de descarga (J/kg)
         Q_dot_comp, # Calor trocado pelo compressor
         h_2, # Entalpia real do ponto de descarga (J/kg)
         T_2, # Temperatura real do ponto de descarga (J/Kg)
         T_3, # Temperatura real na saída do condensador (K)
         h_3, # Entalpia real na saída do condensador (J/kg)
         Q_dot_cond, # Calor trocado pelo condensador
         W_interior, # Umidade real do ambiente interior (Kg agua/Kg ar)
         W_evap, # Umidade real do ar próximo ao evaporador (Kg agua/Kg ar)
         hlv, # Entalpia de vaporização da água na temperatura do evaporador
         Q_dot_evap_sens, # Calor trocado pelo evaporador de forma sensível
         Q_dot_evap_lat, # Calor trocado pelo evaporador de forma latente
         COP] # Coeficiente de performance do sistema
    print(len(X))
    
    # Declaração do vetor com as equações
    F = [P_evap - CP.PropsSI('P','T',T_evap,'Q',1,coolant), 
         P_cond - CP.PropsSI('P','T',T_cond,'Q',0,coolant), 
         eta_v - eta_from_rp_calc(P_cond/P_evap,rot_comp,'v'), 
         eta_g - eta_from_rp_calc(P_cond/P_evap,rot_comp,'g'), 
         T_1 - T_evap - delta_sup, 
         v_1 - 1/CP.PropsSI('D','T',T_1,'P',P_evap,coolant), 
         h_1 - CP.PropsSI('H','T',T_1,'P',P_evap,coolant), 
         s_1 - CP.PropsSI('S','T',T_1,'P',P_evap,coolant), 
         m_dot - eta_v*rot_comp*VOL_cil/v_1, 
         h_2s - CP.PropsSI('H','P',P_cond,'S',s_1,coolant), 
         Pot - m_dot*(h_2s-h_1)/eta_g,
         h_2a - h_1 - Pot/m_dot,
         T_2a - CP.PropsSI('T','P',P_cond,'H',h_2a,coolant),
         Q_dot_comp - UA_comp*(T_2a-T_amb),
         h_2 - h_1 - ((abs(Pot-Q_dot_comp))/m_dot),
         T_2 - CP.PropsSI('T','P',P_cond,'H',h_2,coolant),
         T_3 - T_cond + delta_sub,
         h_3 - CP.PropsSI('H','T',T_3,'P',P_cond,coolant), 
         Q_dot_cond - m_dot*(h_2-h_3), 
         T_cond - T_amb - Q_dot_cond/UA_cond,
         W_interior - psy.GetHumRatioFromTWetBulb(T_int_tbs - CONST_FROM_CELSIUS_TO_K,T_int_tbu - CONST_FROM_CELSIUS_TO_K,ATM_PRESSURE),
         W_evap - psy.GetHumRatioFromRelHum(T_evap - CONST_FROM_CELSIUS_TO_K,1,P_evap),
         hlv - CP.PropsSI('H','T',T_evap,'Q',1,'Water') + CP.PropsSI('H','T',T_evap,'Q',0,'Water'),
         Q_dot_evap_sens - UA_evap_sens*(T_int_tbs - T_evap),
         Q_dot_evap_lat - UA_evap_lat*(W_interior - W_evap)*hlv,
         COP - (Q_dot_evap_sens + Q_dot_evap_lat)/Pot]
    
    # Jacobiana
    
    
    
    # Valor Inicial
    
    
    # NewtonRaphson
    solved = Newton_system(F, J, x, eps)
    
return solved[-1] 